In [3]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier



import ast

In [9]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits =pd.read_csv("tmdb_5000_credits.csv")

ParserError: Error tokenizing data. C error: Expected 20 fields in line 740, saw 24


In [ ]:
movies.head(1)

In [ ]:
movies = movies.drop(['original_title','original_language','production_companies','production_countries','release_date',
                      'spoken_languages','status','homepage','tagline'],axis=1)

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies = movies.dropna()

In [ ]:
movies.isnull().sum()

In [ ]:
credits.head(1)

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L


In [ ]:
credits['crew'] = credits['crew'].apply(fetch_director)

In [ ]:
credits.head(1)

In [ ]:
def convert(text):
  l = []
  for i in ast.literal_eval(text):
    l.append(i['name'])
  return l

In [ ]:
credits['cast'] = credits['cast'].apply(convert)

In [ ]:
credits['cast'] = credits['cast'].apply(lambda x:x[0:3])

In [ ]:
credits.head(1)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies = movies.merge(credits,on="title")

In [ ]:
movies.head(1)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:  x.split())

In [ ]:
def collapse(L):
  L1 = []
  for i in L:
    L1.append(i.replace(" ",""))
  return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies['tags'] = movies['cast'] + movies['crew'] + movies['genres'] + movies['keywords'] + movies['overview']

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [ ]:
movies=movies[['popularity','revenue','runtime','vote_count','budget','title','vote_average','movie_id','tags']]

In [ ]:
movies['vote_average']  = movies['vote_average'].astype("int")

In [ ]:
movies.head(5)

In [ ]:
features = ['popularity','revenue','runtime','vote_count','budget']

In [ ]:
X_train  ,  X_test  , y_train,  y_test = train_test_split(movies[features],movies['vote_average'] , test_size=0.35, random_state=42)

In [ ]:
dt = DecisionTreeClassifier()
gb = GradientBoostingClassifier()
rf = RandomForestClassifier()
xg =XGBClassifier(max_depth=3,
    gamma=4,
    eta=0.3,
    reg_alpha=0.5,
    reg_lambda=0.6)

In [ ]:
model = VotingClassifier(estimators=[('dt',dt),('gb',gb),('rf',rf),('xg',xg)] , voting='hard')
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
model.score(X_train,y_train)

In [ ]:
dt.fit(X_train,y_train)
gb.fit(X_train,y_train)
rf.fit(X_train,y_train)
xg.fit(X_train,y_train)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=4806,stop_words='english')

In [ ]:
vector  = cv.fit_transform(movies['tags'].values.astype('U')).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
def predict(name):
  x = movies.loc[movies.title ==name].iloc[:,0:5]
  fp = (rf.predict(x)+gb.predict(x)+xg.predict(x)+dt.predict(x))/4
  print(f"score prediction averaging {int(fp)}")
  print(f"score prediction max_voting {int(model.predict(x))}")
  print(f"actual score {int(movies.loc[movies.title == name]['vote_average'])}\n")
  print(f"Similar movies for ({name}):")

  index =  movies.loc[movies.title == name].index[0]
  distance = sorted(list(enumerate(similarity[index])), reverse=True,key=lambda vector:vector[1])
  for i in distance[1:5]:
    print(movies.iloc[i[0]]['title'])

In [ ]:
predict("The Dark Knight Rises")